In [67]:
import numpy as np
import pandas as pd
import re

In [68]:
## read data set
data = pd.read_json('D:/Data Science/Course_Quera/Class II/Project2/search_suggestion/data/mrbilit_search.json')
data.head()

,ServiceType,TypedStrings,AcceptString
0,flight,"[, ف, فل, فلو]",فلورانس
1,hotel,"[, H, H,, H, , آ, آو, آوا, آواپ, آواپل, آواپلا...",شیراز
2,hotel,"[, آ, آو, آوا, آواپ, آواپل, آواپلا, آواپلاس]",آواپلاس
3,hotel,[],بین المللی اسپیناس خلیج فارس
4,bus,[],تهران


In [69]:
## read test set
test = pd.read_json('D:/Data Science/Course_Quera/Class II/Project2/search_suggestion/data/test_data.json')
test.head()

,Typed
0,تهد
1,نها
2,ساوه
3,یا
4,بندر انزلی


In [70]:
cities = pd.read_csv('D:/Data Science/Course_Quera/Class II/Project2/search_suggestion/data/iran_cities.csv')
cities.head()

,City EN,City FA,Province,Countise,District,Latitude,Longitude,Area,Elevation,2016 Census,2011 Census,Wikipedia EN,Wikipedia FA,GeoHack
0,Ab Bar,آب‌بر,زنجان,طارم,مرکزی,36.918100,48.964700,709.0,NaN,8091.0,6725.0,https://en.wikipedia.org/wiki/Ab_Bar,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%E2%...,https://geohack.toolforge.org/geohack.php?lang...
1,Ab Pakhsh,آب‌پخش,بوشهر,دشتستان,آب‌پخش,29.363706,51.070180,251204.0,7003.0,18913.0,17238.0,https://en.wikipedia.org/wiki/Ab_Pakhsh,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%E2%...,https://geohack.toolforge.org/geohack.php?lang...
2,Abad,آباد,بوشهر,تنگستان,مرکزی,29.028300,51.249100,45.0,NaN,3787.0,3787.0,"https://en.wikipedia.org/wiki/Abad,_Bushehr",https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...
3,Abadan,آبادان,خوزستان,آبادان,مرکزی,30.339167,48.304167,3.0,NaN,231476.0,212744.0,"https://en.wikipedia.org/wiki/Abadan,_Iran",https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...
4,Abadeh,آباده,فارس,آباده,مرکزی,31.180000,52.670000,1890.0,NaN,59116.0,55758.0,https://en.wikipedia.org/wiki/Abadeh,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...


In [ ]:
# To detect typos due to the English keyboard being active, we were provided with a file called typo_char.csv, which contains English letters and their corresponding Persian letters in a common keyboard layer.

In [71]:
typo = pd.read_csv('D:/Data Science/Course_Quera/Class II/Project2/search_suggestion/data/typo_char.csv')
typo.head()

,EN,FA
0,a,ش
1,b,ذ
2,c,ز
3,d,ی
4,e,ث


In [72]:
## clean columns AcceptString and Typed from unused charackters
def preprocess_text(text):
  
    text = text.split('-')[0]
    text = re.sub(r'\\u[a-fA-F0-9]{4}', '', text)  # حذف کاراکترهای غیر قابل چاپ و نشانه‌های غیر ضروری
    #text = re.sub(r'[^a-zA-Z0-9آ-ی\s]', '', text)  # حذف علایم غیر ضروری ولی حفظ حروف انگلیسی، فارسی و فضاها
    #text = re.sub(r'\s+', ' ', text).strip()  # حذف فضاهای اضافی
    return text

data.AcceptString = data.AcceptString.apply(lambda x: preprocess_text(x))
test.Typed = test.Typed.apply(lambda x: preprocess_text(x))
cities['City FA'] = cities['City FA'].astype(str).apply(lambda x: preprocess_text(x))

In [73]:
## filter data for bus and taxi services
data_zamini = data[(data['ServiceType'] == 'bus') | (data['ServiceType'] == 'taxi')] 
data_zamini.head()

,ServiceType,TypedStrings,AcceptString
4,bus,[],تهران
5,bus,[تهران],تهران
6,bus,"[, ]",اصفهان
7,taxi,[اصفهان],اصفهان
14,bus,[تهران],تهران


In [75]:
## find the most frequented cities
most_frequented_city = data_zamini['AcceptString'].value_counts()[:10].index.tolist()
most_frequented_city

['تهران',
 'اصفهان',
 'مشهد',
 'شیراز',
 'یزد',
 'قم',
 'بندرعباس',
 'رشت',
 'تبریز',
 'کرج']

In [76]:
## write a dictionary of typo and correct charackter from typo file
typo_dict = dict(zip(typo['EN'], typo['FA']))

## replace english charackter with corresponding farsi ones
def correct_typo(text, typo_dict):
    for typo, correct in typo_dict.items():
        text = text.replace(typo, correct)
    return text

## پیش‌پردازش ورودی تست
test['corrected_input'] = test['Typed'].apply(lambda x: correct_typo(x, typo_dict))
test

,Typed,corrected_input
0,تهد,تهد
1,نها,نها
2,ساوه,ساوه
3,یا,یا
4,بندر انزلی,بندر انزلی
...,...,...
514,یز,یز
515,;v,کر
516,بوک,بوک
517,دارا,دارا


In [77]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## use the fuzzy string matching, which uses Levenshtein Distance, to calculate the similarity between sequences

## Step 1
# استخراج لیست شهرها    
city_names = cities['City FA'].tolist()

def suggest_similar_cities(user_input, city_names, n=5):
    #user_input = correct_typo(user_input, typo_dict)
    suggestions = process.extract(user_input, city_names, limit=n)
    return [suggestion[0] for suggestion in suggestions]

test['suggestions_step1'] = test['corrected_input'].apply(lambda x: suggest_similar_cities(x, city_names))

## Step 2
# پرتکرارترین شهرها
def suggest_frequent_cities(n=5):
    return most_frequented_city[:n]

## combine Step 1 and Step 2
def combine_suggestions(suggested_cities_step1, n=5):
    if len(suggested_cities_step1) == n:
        return suggested_cities_step1
    return suggested_cities_step1 + suggest_frequent_cities(n - len(suggested_cities_step1))


## final suggestion
test['final_suggestions'] = test['suggestions_step1'].apply(lambda x: combine_suggestions(x))
test['final_suggestions'] = test['final_suggestions'].astype(str).apply(lambda x: preprocess_text(x))

In [79]:
test[['Suggestion0', 'Suggestion1', 'Suggestion2', 'Suggestion3', 'Suggestion4']] = test['final_suggestions'].str.split(',', expand=True)

In [80]:
submission = pd.DataFrame({'Suggestion0': test['Suggestion0'],
                           'Suggestion1': test['Suggestion1'],
                           'Suggestion2': test['Suggestion2'],
                           'Suggestion3': test['Suggestion3'],
                           'Suggestion4': test['Suggestion4']})

In [81]:
submission.head()

,Suggestion0,Suggestion1,Suggestion2,Suggestion3,Suggestion4
0,['کلاته','کنارتخته','سرخنکلاته','کلاته خیج','باغبهادران']
1,['نهاوند','ارمغانخانه','آشخانه','آستانه','دامنه']
2,['ساوه','پاوه','بندر گناوه','ماسوله','میرجاوه']
3,['آشتیان','ازندریان','بیارجمند','بوئین میاندشت','داریان']
4,['بندر انزلی','انزلی','بن','بندر امام خمینی','داران']
